In [1]:
import numpy as np
import polars as pl
from sklearn.metrics import average_precision_score

In [2]:
# クラス名
class_names = ["猫", "犬", "鳥", "魚"]

# サンプルデータ (10個のサンプル、4つのクラス)
y_true = np.array(
    [
        [1, 0, 1, 0],  # 猫と鳥
        [0, 1, 0, 0],  # 犬
        [1, 1, 0, 0],  # 猫と犬
        [0, 0, 0, 1],  # 魚
        [1, 0, 0, 1],  # 猫と魚
        [0, 1, 1, 0],  # 犬と鳥
        [1, 0, 1, 1],  # 猫と鳥と魚
        [0, 0, 1, 0],  # 鳥
        [1, 1, 0, 1],  # 猫と犬と魚
        [0, 1, 1, 1],  # 犬と鳥と魚
    ]
)

y_scores = np.array(
    [
        [0.9, 0.8, 0.7, 0.1],  # 猫と鳥が正解だが、犬のスコアも高い
        [0.9, 0.7, 0.2, 0.1],  # 犬が正解だが、猫のスコアが最も高い
        [0.8, 0.7, 0.6, 0.2],  # 猫と犬が正解だが、鳥のスコアも高い
        [0.1, 0.2, 0.3, 0.6],  # 魚が正解で、正しく予測
        [0.8, 0.1, 0.7, 0.6],  # 猫と魚が正解だが、鳥のスコアも高い
        [0.2, 0.8, 0.7, 0.1],  # 犬と鳥が正解で、ほぼ正しく予測
        [0.9, 0.2, 0.8, 0.7],  # 猫と鳥と魚が正解で、ほぼ正しく予測
        [0.6, 0.7, 0.5, 0.1],  # 鳥が正解だが、犬のスコアが最も高い
        [0.7, 0.8, 0.2, 0.6],  # 猫と犬と魚が正解だが、鳥のスコアが低い
        [0.1, 0.6, 0.8, 0.7],  # 犬と鳥と魚が正解だが、猫のスコアが低い
    ]
)

参考になりそう
https://fangdahan.medium.com/calculate-mean-average-precision-map-for-multi-label-classification-b082679d31be

In [3]:
# scikit-learnを使用して検証
for i in range(len(class_names)):
    ap_sklearn = average_precision_score(y_true[:, i], y_scores[:, i])
    print(f"{class_names[i]}のAP (scikit-learn): {ap_sklearn:.3f}")

mAP_sklearn = np.mean(
    [
        average_precision_score(y_true[:, i], y_scores[:, i])
        for i in range(len(class_names))
    ]
)
print(f"\nmAP (scikit-learn): {mAP_sklearn:.3f}")

猫のAP (scikit-learn): 0.753
犬のAP (scikit-learn): 0.676
鳥のAP (scikit-learn): 0.863
魚のAP (scikit-learn): 1.000

mAP (scikit-learn): 0.823


In [4]:
ap_list = []
for class_idx, class_name in enumerate(class_names):
    class_df = pl.DataFrame(
        {
            "y_true": y_true[:, class_idx],
            "y_scores": y_scores[:, class_idx],
        }
    ).sort("y_scores", descending=True)
    ap = 0
    prev_recall = None
    for i in range(len(class_df)):
        precision = np.sum(class_df["y_true"][: i + 1].to_numpy()) / (i + 1)
        recall = np.sum(class_df["y_true"][: i + 1].to_numpy()) / np.sum(
            class_df["y_true"].to_numpy()
        )
        if i == 0:
            ap += precision * recall
        else:
            ap += precision * (recall - prev_recall)
        prev_recall = recall
        # recall が1.0になったら終了
        if recall == 1.0:
            break

    print(f"{class_name}のAP: {ap:.3f}")
    ap_list.append(ap)

mAP = np.mean(ap_list)
print(f"mAP: {mAP:.3f}")

猫のAP: 0.810
犬のAP: 0.686
鳥のAP: 0.903
魚のAP: 1.000
mAP: 0.850
